In [25]:
import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import pickle
from glob import glob
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import random_split
import pandas as pd
import time
from einops import rearrange,reduce,repeat
from tqdm import trange

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [26]:
class MyLSTM(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(MyLSTM, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        #Initializing hidden state for first input using method defined below
        h_t, c_t = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, (h_t, c_t) = self.lstm(x, (h_t, c_t))
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        #out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, (h_t, c_t)
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        h_0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
        c_0 =  torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)       
         # We'll send the tensor holding the hidden state to the device we specified earlier as well
        return h_0, c_0

In [27]:
class PVDataset(torch.utils.data.Dataset):
    def __init__(self, 
                 x_data_path,
                 y_data_path):
        super(PVDataset, self).__init__()
        
        self.X = np.load(x_data_path)
        self.y = np.load(y_data_path)

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        sample_x = self.X[idx]
        sample_y = self.y[idx]
        return torch.from_numpy(sample_x).float(), torch.from_numpy(sample_y).float()

In [28]:
batch_size = 1024
in_dim = 19*2
out_dim = 30*2
hidden_dim = 80
num_layers = 3
learning_rate = 0.01
decay_rate = 0.95
num_epoch = 1000
name = 'rnn_fang_clean_3'
model_name = name + '_model.pth'
submission_name = name + '_submission.csv'

In [29]:
model = MyLSTM(input_size=in_dim, output_size=out_dim, hidden_dim=hidden_dim, n_layers=num_layers).to(device)   #maximum number of hidden size is 120



optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=decay_rate)  # stepwise learning rate decay
loss_fun = nn.MSELoss()

In [30]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def train_epoch(train_loader, model, optimizer, loss_function):
    model.train()
    train_mse = []
    for inp, tgt in train_loader:
        inp = inp.to(device)
        tgt = tgt.to(device)
       
        
        pred,_ = model(inp)
        #print(pred.shape,tgt.shape)
        pred = pred[:,-1, :].squeeze(1)
        loss = loss_function(pred, tgt)
        train_mse.append(loss.item()) 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_mse = round(np.sqrt(np.mean(train_mse)),5)
    return train_mse

def eval_epoch(valid_loader, model, loss_function):
    model.eval()
    valid_mse = []
    with torch.no_grad():
        for inp, tgt in valid_loader:
            inp = inp.to(device)
            tgt = tgt.to(device)
            
            loss = 0
            pred,_ = model(inp)
            #print(tgt.shape,pred.shape)
            pred = pred[:,-1, :].squeeze(1)
            loss = loss_function(pred, tgt)
            valid_mse.append(loss.item())
            
        valid_mse = round(np.sqrt(np.mean(valid_mse)), 5)
    return valid_mse

In [31]:
train_rmse = []
valid_rmse = []
min_rmse = 10e8

dataset = PVDataset('train_X_clean.npy', 'train_y_clean.npy')

for epoch in range(num_epoch):
    start = time.time()
    train_set, valid_set = random_split(
        dataset=dataset,
        lengths=[0.9, 0.1],
        generator=torch.Generator().manual_seed(0)
    )

    # create dataloaders
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size, shuffle=False)

    model.train() # if you use dropout or batchnorm. 
    train_rmse.append(train_epoch(train_loader, model, optimizer, loss_fun))

    model.eval()
    valid_rmse.append(eval_epoch(valid_loader, model, loss_fun))

    # save the best model
    if valid_rmse[-1] < min_rmse:
        min_rmse = valid_rmse[-1] 
        best_model = model
        torch.save([best_model, epoch, get_lr(optimizer)], model_name)

    end = time.time()
    
    # Early Stopping
    if (len(train_rmse) > 100 and np.mean(valid_rmse[-5:]) >= np.mean(valid_rmse[-10:-5])):
            break       

    # Learning Rate Decay        
    scheduler.step()
    
    print("Epoch {} | T: {:0.2f} | Train RMSE: {:0.5f} | Valid RMSE: {:0.5f}".format(epoch + 1, (end-start) / 60, train_rmse[-1], valid_rmse[-1]))

Epoch 1 | T: 0.06 | Train RMSE: 0.19090 | Valid RMSE: 0.07614
Epoch 2 | T: 0.06 | Train RMSE: 0.06057 | Valid RMSE: 0.05820
Epoch 3 | T: 0.06 | Train RMSE: 0.04568 | Valid RMSE: 0.04870
Epoch 4 | T: 0.05 | Train RMSE: 0.03776 | Valid RMSE: 0.03545
Epoch 5 | T: 0.06 | Train RMSE: 0.03433 | Valid RMSE: 0.03505
Epoch 6 | T: 0.05 | Train RMSE: 0.03401 | Valid RMSE: 0.03525
Epoch 7 | T: 0.05 | Train RMSE: 0.03297 | Valid RMSE: 0.03563
Epoch 8 | T: 0.05 | Train RMSE: 0.03252 | Valid RMSE: 0.03214
Epoch 9 | T: 0.06 | Train RMSE: 0.03199 | Valid RMSE: 0.03382
Epoch 10 | T: 0.05 | Train RMSE: 0.03206 | Valid RMSE: 0.03570
Epoch 11 | T: 0.06 | Train RMSE: 0.03131 | Valid RMSE: 0.03159
Epoch 12 | T: 0.05 | Train RMSE: 0.03124 | Valid RMSE: 0.03120
Epoch 13 | T: 0.05 | Train RMSE: 0.03116 | Valid RMSE: 0.03233
Epoch 14 | T: 0.05 | Train RMSE: 0.03051 | Valid RMSE: 0.03152
Epoch 15 | T: 0.05 | Train RMSE: 0.03084 | Valid RMSE: 0.03125
Epoch 16 | T: 0.05 | Train RMSE: 0.03010 | Valid RMSE: 0.03422
E

In [32]:
best_model, _, _ = torch.load(model_name)
best_model

MyLSTM(
  (lstm): LSTM(38, 80, num_layers=2, batch_first=True)
  (fc): Linear(in_features=80, out_features=60, bias=True)
)

In [33]:
def replace_bad_data_points(positions, velocities, total_distance):
    if total_distance < 10:
        return positions, velocities
    
    positions_copy = np.copy(positions)
    positions_copy[1:,:] = positions[:-1,:]
    distances = np.linalg.norm(positions-positions_copy, axis=1)
    bad_indices = np.where(distances[1:-1] == 0)[0]+1

    if len(bad_indices) == 0:
        return positions, velocities

    positions_copy = np.copy(positions)
    velocities_copy = np.copy(velocities)
    for index in bad_indices:
        positions_copy[index] = (positions[index - 1] + positions[index + 1]) / 2

    bad_indices = np.where(velocities[1:-1] == 0)[0]+1
    for index in bad_indices:
        velocities_copy[index] = (velocities[index - 1] + velocities[index + 1]) / 2

    return positions_copy, velocities_copy

def detect_outlier(folder, file_name, thresh, agent_only=True, verbose=True):
    with open(os.path.join(folder, file_name + '.pkl'), 'rb') as f:
        data = pickle.load(f)

    num_cars = data['car_mask'].astype(int).sum()
    if 'p_out' in data:
        car_points = np.concatenate((data['p_in'], data['p_out']), axis=1)
        velocities = np.concatenate((data['v_in'], data['v_out']), axis=1)
    else:
        car_points = data['p_in']
        velocities = data['v_in']
    car_points = car_points[:num_cars, ...]
    
    if agent_only:
        agent_idx = np.where(data['track_id'] == data['agent_id'])[0][0]
        car_points = car_points[agent_idx]
    
    if agent_only:
        axis1=1
        axis2=0
        total_distance = np.linalg.norm(car_points[0]-car_points[-1])
        car_points, _ = replace_bad_data_points(car_points, velocities, total_distance)
    else:
        axis1=2
        axis2=1
        total_distance = np.linalg.norm(car_points[:,0]-car_points[:,-1], axis=axis2)
        for i in range(car_points.shape[0]):
            car_points[i], _ = replace_bad_data_points(car_points[i], velocities[i], total_distance[i])

    car_points_copy = np.copy(car_points)
    if agent_only:
        car_points_copy[1:,:] = car_points[:-1,:]
    else:
        car_points_copy[:,1:,:] = car_points[:,:-1,:]
    distances = np.linalg.norm(car_points-car_points_copy, axis=axis1)
    prev_idx = np.argmax(distances, axis=axis2)

    if agent_only:
        prev = distances[prev_idx-1]
        if verbose:
            print((prev))
            print(np.max(distances, axis=axis2),  np.median(distances, axis=axis2))
        return np.max(distances, axis=axis2) > thresh * prev
    else:
        prev_idx = np.array([[i, idx-1] for i,idx in enumerate(prev_idx)])
        prev = distances[prev_idx[:, 0], prev_idx[:, 1]]
        if verbose:
            print(velocities[5])
            print(np.max(distances, axis=axis2),  np.median(distances, axis=axis2))
        for i,d in enumerate(distances):
            total_d = total_distance[i]
            if total_d > 5 and np.max(d) > thresh * prev[i] and prev_idx[i, 1] != 0:
                return True
        
        return False
        


In [34]:
test_path = "./val_in/val_in/"
test_pkl_list = glob(os.path.join(test_path, '*'))
test_pkl_list.sort()

test_preds = []
for idx in range(len(test_pkl_list)):
    with open(test_pkl_list[idx], 'rb') as f:
        test_sample = pickle.load(f)
        pred_id = np.where(test_sample["track_id"] == test_sample['agent_id'])[0][0]
        mask = np.where(test_sample['car_mask'] == 1)[0]
        # input: p_in & v_in; output: p_out
        
        p_in_raw = test_sample['p_in']
        #p_out_raw = test_sample['p_out'][pred_id]
        v_in_raw = test_sample['v_in']
        lane_scene = test_sample['lane']
        
        # Normalization
        min_vecs = np.min(lane_scene, axis = 0)
        max_vecs = np.max(lane_scene, axis = 0)
        
        # Normalize by vectors
        p_in_norm = (p_in_raw - min_vecs)/(max_vecs - min_vecs)
        
        v_in_norm = np.linalg.norm(v_in_raw, axis=1, keepdims=True)
        v_in_norm = np.where(v_in_norm == 0.0, 1.0, v_in_norm)
        v_in_norm = v_in_raw / v_in_norm
        # v_out_normalized = v_out_raw / np.linalg.norm(v_out_raw, axis=1, keepdims=True)
        p_track = p_in_norm[mask].reshape(-1,19*2)
        v_track = v_in_norm[mask].reshape(-1,19*2)
        
        p_agent = p_in_norm[pred_id].reshape(1,-1)
        v_agent = v_in_norm[pred_id].reshape(1,-1)
        new_mask = []
        p_result = []
        v_result = []
        if (len(mask) > 10):
            
            dist = ((p_track - p_agent)**2).sum(axis=-1)
            #print('dist is ',dist.shape)
            new_mask = np.argpartition(dist,10)[:10]
            
            p_result = p_track[new_mask,:]
            v_result = v_track[new_mask,:]
        else:
            p_result = np.zeros((10,38))
            v_result = np.zeros((10,38))
            k = p_track.shape[0]
            #print('slice',k)
            p_result[:k] = p_track
            v_result[:k] = v_track
        
        #print('shape is',p_agent.shape,v_agent.shape,p_result.shape,v_result.shape)
        inp = np.vstack((p_agent,v_agent,p_result,v_result)).reshape(1,22,38)
        inp = torch.from_numpy(inp).to(device,dtype=torch.float)
        preds,_ = model(inp)
        preds = preds.squeeze(0)
        pred = preds[-1, :].cpu().data.numpy()
        pred = rearrange(pred, "(b c) -> b c", c =2 )
        # De-Normalization ! 
        pred = pred * (max_vecs[:2] - min_vecs[:2]) +  min_vecs[:2]
        test_preds.append(pred)

In [39]:
# # Submission Files
sample_sub = pd.read_csv('sample_submission.csv')

# Convert to int
predictions = np.concatenate(test_preds).reshape(len(test_preds), -1)
sub_df = pd.DataFrame(np.c_[sample_sub["ID"], predictions], columns=[np.r_[["ID"], ["v" + str(i) for i in range(1, 61)]]])
sub_df['ID'] = sub_df['ID'].astype('int')
sub_df.to_csv(submission_name, index=None)
sub_df.dtypes

ID       int32
v1     float64
v2     float64
v3     float64
v4     float64
        ...   
v56    float64
v57    float64
v58    float64
v59    float64
v60    float64
Length: 61, dtype: object